### Objective:
Create a code that does QR decomposition. where Q is an orthogonal matrix and R is an upper triangular matrix, and performs column pivoting.

In [1]:
import numpy as np

In [2]:
def QR_decomposition(A, eps = 1e-4):
    
    # Initialization
    A_ref = np.copy(A)
    row, col = np.shape(A_ref)
    if row > col:
        Q = np.zeros([row, col], dtype=float)
        R = np.zeros([col, col], dtype=float)
    elif row < col:
        Q = np.zeros([row, row], dtype=float)
        R = np.zeros([row, col], dtype=float)
    else:
        Q = np.zeros([row, col], dtype=float)
        R = np.zeros([row, col], dtype=float)
    
    P = np.diag(np.ones(col)) 
    pivot_row = 0
    iterate = 0
    
    for pivot_col in range(col):
        
        # Column Pivoting #by magnitude based on error??
        max_col = np.argmax(abs(A_ref[pivot_row, pivot_col:])) + pivot_col 
        if pivot_col != max_col and pivot_col < row-1:
            P[:,[pivot_col, max_col]] = P[:,[max_col, pivot_col]]
            R[:,[pivot_col, max_col]] = R[:,[max_col, pivot_col]]
            A_ref[:,[pivot_col, max_col]] = A_ref[:,[max_col, pivot_col]]
        
        # Skips for columns with (eps,-eps)
        while np.all(np.abs(A_ref[:, pivot_col]) < eps):
            continue
        
        # QR Decomposition
        e_vector = np.copy(A_ref[:, pivot_col]) # copy vector a to later get vector e
        if iterate == 0: # just for 1st iteration
            e_magnitude = np.linalg.norm(e_vector) # get magnitude of vector e
            R[pivot_row][pivot_col] = e_magnitude
            Q[:,pivot_col] = e_vector/e_magnitude
            # divide vector e by its magnitude to get unit vector q => q = e/||e||
            iterate += 1
        else:
            for r_row in range(pivot_row+1): 
                q_col = r_row
                QTA = np.dot(Q[:,q_col], A_ref[:, pivot_col]) 
                # vector q transpose times vector a => (q.T)a = qdota
                R[r_row][pivot_col] = QTA
                e_vector -= Q[:,q_col]*QTA 
                # project vector a to vector q to get vector p => p = q(q.T)a
                # subtract vector p from vector a to get vector e  => e = a - q(q.T)a
            e_magnitude = np.linalg.norm(e_vector) # get magnitude of vector e
            if pivot_col < row:
                R[pivot_row, pivot_col] = e_magnitude 
                Q[:,pivot_col] = e_vector/e_magnitude 
                # divide vector e by its magnitude to get unit vector q => q = e/||e||
        
        if pivot_row < (row-1):
            pivot_row += 1 
            
    return Q, R, P

In [3]:
def show_matrix(A, eps = 1e-4):
    Q, R, P = QR_decomposition(A)
    AmulP = np.matmul(A,P)
    QmulR = np.matmul(Q,R)
    QT = Q.transpose()
    QQTA = (Q.dot(QT)).dot(A)
    print('Is AP = QR? {}'.format(np.allclose(AmulP, QmulR, rtol=eps)))
    print('AP:\n{}\nQR:\n{}\n'
          .format(np.matrix.round(AmulP, decimals = 2), np.matrix.round(QmulR, decimals = 2)))
    print('Is Matrix Q orthonormal (A = QQTA)? {}'
          .format(np.allclose(A, QQTA, rtol=eps)))
    print('QQTA:\n{}\n'
          .format(np.matrix.round(QQTA, decimals = 2)))
    print('Individual Matrices:\n\nMatrix A:\n{}\n\nMatrix P:\n{}\n\nMatrix Q:\n{}\n\nMatrix R:\n{}'
          .format(np.matrix.round(A, decimals = 2), np.matrix.round(P, decimals = 2), 
                  np.matrix.round(Q, decimals = 2), np.matrix.round(R, decimals = 2)))

In [4]:
def for_rand_matrix(row, col, iteration = 1000, eps = 1e-10):
    eq_correct = 0
    ortho_correct = 0

    for i in range(iteration):
        A = np.random.randn(row,col)

        Q, R, P = QR_decomposition(A)
        AmulP = np.matmul(A,P)
        QmulR = np.matmul(Q,R)
        QT = Q.transpose()
        QQTA = (Q.dot(QT)).dot(A)

        check_eq = np.allclose(AmulP, QmulR, rtol=eps)
        check_ortho = np.allclose(A, QQTA, rtol=eps)

        eq_correct += 1 if check_eq else 0
        ortho_correct += 1 if check_ortho else 0

    print(f'For a {row}x{col} matrix:')
    print(f'{eq_correct} out of {iteration} iterations are AP = QR.')
    print(f'{ortho_correct} out of {iteration} iterations are A = QQTA.')

**Example 1:** row = column

In [5]:
A1 = np.array([[ 3,  2,  1],
               [ 1, -1,  1],
               [ 0,  1, -1]], float)
show_matrix(A1)

Is AP = QR? True
AP:
[[ 3.  2.  1.]
 [ 1. -1.  1.]
 [ 0.  1. -1.]]
QR:
[[ 3.  2.  1.]
 [ 1. -1.  1.]
 [ 0.  1. -1.]]

Is Matrix Q orthonormal (A = QQTA)? True
QQTA:
[[ 3.  2.  1.]
 [ 1. -1.  1.]
 [-0.  1. -1.]]

Individual Matrices:

Matrix A:
[[ 3.  2.  1.]
 [ 1. -1.  1.]
 [ 0.  1. -1.]]

Matrix P:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Matrix Q:
[[ 0.95  0.27  0.17]
 [ 0.32 -0.8  -0.51]
 [ 0.    0.53 -0.85]]

Matrix R:
[[ 3.16  1.58  1.26]
 [ 0.    1.87 -1.07]
 [ 0.    0.    0.51]]


**Example 2:** row < column

In [6]:
A2 = np.array([[ 3,  2,  1],
               [ 1, 4,  1]], float)
show_matrix(A2)

Is AP = QR? True
AP:
[[3. 2. 1.]
 [1. 4. 1.]]
QR:
[[3. 2. 1.]
 [1. 4. 1.]]

Is Matrix Q orthonormal (A = QQTA)? True
QQTA:
[[3. 2. 1.]
 [1. 4. 1.]]

Individual Matrices:

Matrix A:
[[3. 2. 1.]
 [1. 4. 1.]]

Matrix P:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Matrix Q:
[[ 0.95 -0.32]
 [ 0.32  0.95]]

Matrix R:
[[3.16 3.16 1.26]
 [0.   3.16 0.63]]


**Example 3:** row > column

In [7]:
A2 = np.array([[ 1,  3],
               [ 2,  2],
               [ 4, 12]], float)
show_matrix(A2)

Is AP = QR? True
AP:
[[ 3.  1.]
 [ 2.  2.]
 [12.  4.]]
QR:
[[ 3.  1.]
 [ 2.  2.]
 [12.  4.]]

Is Matrix Q orthonormal (A = QQTA)? True
QQTA:
[[ 1.  3.]
 [ 2.  2.]
 [ 4. 12.]]

Individual Matrices:

Matrix A:
[[ 1.  3.]
 [ 2.  2.]
 [ 4. 12.]]

Matrix P:
[[0. 1.]
 [1. 0.]]

Matrix Q:
[[ 0.24 -0.04]
 [ 0.16  0.99]
 [ 0.96 -0.15]]

Matrix R:
[[12.53  4.39]
 [ 0.    1.32]]


**Example 4:** 0 at start

In [8]:
A4 = np.array([[ 0,  5,  6,  1],
               [ 1,  2,-12, -1],
               [ 4, -2, 12,  5],
               [ 2,  0,  0,  3]], float)
show_matrix(A4)

Is AP = QR? True
AP:
[[  6.   5.   1.   0.]
 [-12.   2.  -1.   1.]
 [ 12.  -2.   5.   4.]
 [  0.   0.   3.   2.]]
QR:
[[  6.   5.   1.  -0.]
 [-12.   2.  -1.   1.]
 [ 12.  -2.   5.   4.]
 [  0.   0.   3.   2.]]

Is Matrix Q orthonormal (A = QQTA)? True
QQTA:
[[ -0.   5.   6.   1.]
 [  1.   2. -12.  -1.]
 [  4.  -2.  12.   5.]
 [  2.   0.   0.   3.]]

Individual Matrices:

Matrix A:
[[  0.   5.   6.   1.]
 [  1.   2. -12.  -1.]
 [  4.  -2.  12.   5.]
 [  2.   0.   0.   3.]]

Matrix P:
[[0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]

Matrix Q:
[[ 0.33  0.94  0.   -0.  ]
 [-0.67  0.24  0.49  0.51]
 [ 0.67 -0.24  0.49  0.51]
 [ 0.    0.    0.73 -0.69]]

Matrix R:
[[18.   -1.    4.33  2.  ]
 [ 0.    5.66 -0.47 -0.71]
 [ 0.    0.    4.12  3.88]
 [ 0.    0.    0.    1.2 ]]


**Example 5:** Random 100x100 matrix

In [9]:
Ax = np.random.randn(100,100)
show_matrix(Ax)

Is AP = QR? True
AP:
[[-3.37 -0.46  1.07 ... -1.1  -0.69 -1.58]
 [ 0.15 -2.69  2.23 ...  0.49 -0.61  1.26]
 [-2.68  0.99  2.83 ... -1.81  0.02 -0.41]
 ...
 [ 0.73 -0.11 -1.2  ... -0.59  0.2  -0.38]
 [ 0.12 -0.72  0.92 ... -0.54 -0.29  0.22]
 [-0.02  0.35  1.31 ... -0.57 -0.04  1.44]]
QR:
[[-3.37 -0.46  1.07 ... -1.1  -0.69 -1.58]
 [ 0.15 -2.69  2.23 ...  0.49 -0.61  1.26]
 [-2.68  0.99  2.83 ... -1.81  0.02 -0.41]
 ...
 [ 0.73 -0.11 -1.2  ... -0.59  0.2  -0.38]
 [ 0.12 -0.72  0.92 ... -0.54 -0.29  0.22]
 [-0.02  0.35  1.31 ... -0.57 -0.04  1.44]]

Is Matrix Q orthonormal (A = QQTA)? True
QQTA:
[[ 0.32 -0.34 -0.65 ... -1.96 -0.58  1.88]
 [ 1.08  1.06  0.03 ...  0.34 -0.88  0.08]
 [ 0.79  1.48  0.28 ...  1.84 -1.14 -1.41]
 ...
 [-1.98 -0.05  0.91 ... -1.79 -0.69 -0.05]
 [-0.53 -1.19  0.15 ...  0.92  0.85 -0.08]
 [ 1.58  0.48  0.8  ...  0.18 -0.21  0.6 ]]

Individual Matrices:

Matrix A:
[[ 0.32 -0.34 -0.65 ... -1.96 -0.58  1.88]
 [ 1.08  1.06  0.03 ...  0.34 -0.88  0.08]
 [ 0.79  1.48  0

**Example 6:** Random 100x50 matrix

In [10]:
Ax = np.random.randn(100,50)
show_matrix(Ax)

Is AP = QR? True
AP:
[[ 2.38 -1.35 -0.64 ... -0.02 -0.48  0.6 ]
 [-1.01 -2.57  1.09 ...  2.48  1.24 -0.76]
 [ 1.44  0.83 -2.95 ...  0.14 -1.6   1.02]
 ...
 [-0.61  1.    1.27 ... -0.62 -0.86 -0.54]
 [-1.56 -2.09 -0.94 ...  0.35  0.11  0.01]
 [ 1.65  0.52 -0.33 ... -0.1   0.96  0.3 ]]
QR:
[[ 2.38 -1.35 -0.64 ... -0.02 -0.48  0.6 ]
 [-1.01 -2.57  1.09 ...  2.48  1.24 -0.76]
 [ 1.44  0.83 -2.95 ...  0.14 -1.6   1.02]
 ...
 [-0.61  1.    1.27 ... -0.62 -0.86 -0.54]
 [-1.56 -2.09 -0.94 ...  0.35  0.11  0.01]
 [ 1.65  0.52 -0.33 ... -0.1   0.96  0.3 ]]

Is Matrix Q orthonormal (A = QQTA)? True
QQTA:
[[-0.49  0.48 -0.88 ...  0.59  0.08  0.79]
 [ 0.12 -1.49  1.12 ...  0.01  0.68 -0.22]
 [ 1.64  1.23  1.35 ... -0.39 -1.48  0.86]
 ...
 [ 0.02 -2.58 -1.33 ... -0.68 -0.14 -0.09]
 [-1.37 -0.72  2.09 ... -1.15 -0.45 -0.38]
 [-0.38 -1.56  0.11 ...  0.1  -0.5  -0.33]]

Individual Matrices:

Matrix A:
[[-0.49  0.48 -0.88 ...  0.59  0.08  0.79]
 [ 0.12 -1.49  1.12 ...  0.01  0.68 -0.22]
 [ 1.64  1.23  1

**Example 7:** Random 50x100 matrix

In [11]:
Ax = np.random.randn(50,100)
show_matrix(Ax)

Is AP = QR? True
AP:
[[ 2.79  1.05 -0.6  ... -1.14 -0.24  0.73]
 [-0.67 -3.36 -0.16 ...  1.04 -0.79 -1.16]
 [ 1.3  -1.34 -2.55 ... -0.38  1.03 -0.03]
 ...
 [-0.59 -0.81 -0.61 ... -0.72  0.15 -0.75]
 [ 0.16 -1.09 -0.62 ...  0.78 -1.07  0.38]
 [ 0.28 -0.7  -0.95 ... -2.43 -1.24  0.25]]
QR:
[[ 2.79  1.05 -0.6  ... -1.14 -0.24  0.73]
 [-0.67 -3.36 -0.16 ...  1.04 -0.79 -1.16]
 [ 1.3  -1.34 -2.55 ... -0.38  1.03 -0.03]
 ...
 [-0.59 -0.81 -0.61 ... -0.72  0.15 -0.75]
 [ 0.16 -1.09 -0.62 ...  0.78 -1.07  0.38]
 [ 0.28 -0.7  -0.95 ... -2.43 -1.24  0.25]]

Is Matrix Q orthonormal (A = QQTA)? True
QQTA:
[[ 1.12 -0.22  0.66 ...  0.07  2.79 -0.63]
 [-2.72  0.48 -0.09 ... -0.02 -0.67  1.24]
 [ 0.25  0.16 -0.9  ... -0.91  1.3   1.53]
 ...
 [ 1.72 -0.83 -1.05 ... -1.02 -0.59  0.37]
 [ 2.54 -0.49  0.89 ...  0.57  0.16 -2.53]
 [ 0.78  0.49 -0.35 ... -0.61  0.28  1.24]]

Individual Matrices:

Matrix A:
[[ 1.12 -0.22  0.66 ...  0.07  2.79 -0.63]
 [-2.72  0.48 -0.09 ... -0.02 -0.67  1.24]
 [ 0.25  0.16 -0

**Example 8:** For loop of 5x5 matrix

In [12]:
for_rand_matrix(5,5)

For a 5x5 matrix:
1000 out of 1000 iterations are AP = QR.
1000 out of 1000 iterations are A = QQTA.


**Example 9:** For loop of 5x10 matrix

In [13]:
for_rand_matrix(5,10)

For a 5x10 matrix:
1000 out of 1000 iterations are AP = QR.
1000 out of 1000 iterations are A = QQTA.


**Example 10:** For loop of 10x5 matrix

In [14]:
for_rand_matrix(10,5)

For a 10x5 matrix:
1000 out of 1000 iterations are AP = QR.
1000 out of 1000 iterations are A = QQTA.
